# Basic Search

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---
                                                                    
Before we do any work, we need to import these functions from cdapython. We're also importing functions from several other packages to make viewing and manipulating tables easier. The `opt.` settings are pre-configuring how itables should display our tables, with scrolling and paging enabled.
Finally, we're telling cdapython to report it's version so we can be sure we're using the one we mean to:

In [1]:
from cdapython import Q, columns, unique_terms, query
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)
import itables.options as opt
opt.maxBytes=0
opt.scrollX="200px"
opt.scrollCollapse=True
opt.paging=True
opt.maxColumns=0
print(Q.get_version())

Q.set_host_url('https://cancerdata.dsde-dev.broadinstitute.org/')

<IPython.core.display.Javascript object>

2022.9.15

<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">
    
CDA data comes from three sources:
<ul>
<li><b>The <a href="https://proteomic.datacommons.cancer.gov/pdc/"> Proteomic Data Commons</a> (PDC)</b></li>
<li><b>The <a href="https://gdc.cancer.gov/">Genomic Data Commons</a> (GDC)</b></li>
<li><b>The <a href="https://datacommons.cancer.gov/repository/imaging-data-commons">Imaging Data Commons</a> (IDC)</b></li>
</ul> 
    
The CDA makes this data searchable in five main endpoints:

<ul>
<li><b>subject:</b> A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</li>
<li><b>researchsubject:</b> A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs.</li>
<li><b>specimen:</b> Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</li>
<li><b>file:</b> A unit of data about subjects, researchsubjects, specimens, or their associated information.</li>
<li><b>mutation:</b> Molecular data about specific mutations, currently limited to the TCGA-READ project from GDC.</li>
</ul>
and two endpoints that offer deeper information about data in the researchsubject endpoint:
<ul>
<li><b>diagnosis:</b> A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</li>
<li><b>treatment:</b> Represent medication administration or other treatment types.</li>
</ul>
</div>


If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specific format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, start with `file`.

In CDA search, concepts can also be chained together, so you can look specifically for specimen files. In the five 'main' tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in specimen files. Diagnosis and treatment do not have files directly associated with them and so can only be used to find files in conjunction with the other searches.

Any metadata field can be searched from any endpoint, the only difference between search types is what type of data is returned by default. This means that you can think of the CDA as a really, really enormous spreadsheet full of data. To search this enormous spreadsheet, you'd want select columns, and then filter rows.



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` where the syntax is `Q('<name_of_column> = "<unique_term>"'` and save it to a variable `myquery`:

In [2]:
myquery = Q('primary_diagnosis_site = "brain"')


<div class="cdawarn" style="background-color:#f9cfbf;color:black;padding:20px;">
    
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [3]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 0
                        min 4.026 sec 4026 ms

We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [4]:
subjectresults


            
            Offset: 0
            Count: 100
            Total Row Count: 2384
            More pages: True
            

This tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---

In any results variable, you can also use `print()` to display a 'pretty print' version of the results. This color codes the display in a way that can make it easier to find the information you care about:


In [5]:
print(subjectresults)

Offset: 0
            Count: 100
            Total Row Count: 2384
            More pages: True

We have 2384 subject matches, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [6]:
subjectresults.to_dataframe()

subject_id                                 subject_identifier  \
0              900-00-5393        [{'system': 'IDC', 'value': '900-00-5393'}]   
1   ACRIN-DSC-MR-Brain-086  [{'system': 'IDC', 'value': 'ACRIN-DSC-MR-Brai...   
2                   C29151             [{'system': 'PDC', 'value': 'C29151'}]   
3        GENIE-DFCI-009743  [{'system': 'GDC', 'value': 'GENIE-DFCI-009743'}]   
4                  LGG-337            [{'system': 'IDC', 'value': 'LGG-337'}]   
..                     ...                                                ...   
95            TCGA-76-6660  [{'system': 'GDC', 'value': 'TCGA-76-6660'}, {...   
96            TCGA-DU-6393  [{'system': 'GDC', 'value': 'TCGA-DU-6393'}, {...   
97            TCGA-HT-7474  [{'system': 'GDC', 'value': 'TCGA-HT-7474'}, {...   
98            TCGA-HT-7873  [{'system': 'GDC', 'value': 'TCGA-HT-7873'}, {...   
99            TCGA-QH-A6X5  [{'system': 'GDC', 'value': 'TCGA-QH-A6X5'}, {...   

         species     sex   race               ethnicity  days_to_birth  \
0   homo sapiens    None   None                    None            NaN   
1   homo sapiens    None   None                    None            NaN   
2   homo sapiens    male  white  not hispanic or latino            NaN   
3   homo sapiens  female  white  not hispanic or latino       -16436.0   
4   homo sapiens    None   None                    None            NaN   
..           ...     ...    ...                     ...            ...   
95  homo sapiens    male  white  not hispanic or latino       -26913.0   
96  homo sapiens    male  white  not hispanic or latino       -24282.0   
97  homo sapiens    male  white  not hispanic or latino       -19149.0   
98  homo sapiens    male  white  not hispanic or latino       -10805.0   
99  homo sapiens  female  white  not hispanic or latino       -21239.0   

                           subject_associated_project  vital_status  \
0                                         [rembrandt]          None   
1                                [acrin_dsc_mr_brain]          None   
2   [Proteogenomic Analysis of Pediatric Brain Can...         Alive   
3                                        [GENIE-DFCI]  Not Reported   
4                                 [lgg_1p19qdeletion]          None   
..                                                ...           ...   
95                               [tcga_gbm, TCGA-GBM]          Dead   
96                               [TCGA-LGG, tcga_lgg]          Dead   
97                               [TCGA-LGG, tcga_lgg]         Alive   
98                               [TCGA-LGG, tcga_lgg]         Alive   
99                               [TCGA-LGG, tcga_lgg]         Alive   

    days_to_death cause_of_death  
0             NaN           None  
1             NaN           None  
2             NaN   Not Reported  
3             NaN           None  
4             NaN           None  
..            ...            ...  
95          114.0           None  
96         1585.0           None  
97            NaN           None  
98            NaN           None  
99            NaN           None  

[100 rows x 11 columns]

By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any [pandas dataframe functions](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs:


In [7]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", na=False)]

subject_id                                 subject_identifier  \
28  TCGA-06-0394  [{'system': 'GDC', 'value': 'TCGA-06-0394'}, {...   
42       C579945            [{'system': 'PDC', 'value': 'C579945'}]   

         species     sex                       race               ethnicity  \
28  homo sapiens    male  black or african american  not hispanic or latino   
42  homo sapiens  female  black or african american  not hispanic or latino   

    days_to_birth                         subject_associated_project  \
28       -18913.0                               [tcga_gbm, TCGA-GBM]   
42            NaN  [Proteogenomic Analysis of Pediatric Brain Can...   

   vital_status  days_to_death cause_of_death  
28         Dead          329.0           None  
42        Alive            NaN   Not Reported

There are subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. 

We'll cover how to work with large results dataframes in the <a href="../Pagination">Pagination</a> notebook. Or, learn how to get summary information from search results in the <a href="../DataSummaries">Data Summaries</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</i>

    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system."</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>species:</b> The taxonomic group (e.g. species) of the subject.</li>
<li><b>sex:</b> The biologic character or quality that distinguishes male and female from one another as expressed by analysis of the person's gonadal, morphologic (internal and external), chromosomal, and hormonal characteristics.</li>
<li><b>race:</b> An arbitrary classification of a taxonomic group that is a division of a species. It usually arises as a consequence of geographical isolation within a species and is characterized by shared heredity, physical attributes and behavior, and in the case of humans, by common history, nationality, or geographic distribution. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>ethnicity:</b> An individual's self-described social and cultural grouping, specifically whether an individual describes themselves as Hispanic or Latino. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days.</li>
<li><b>subject_associated_project:</b> The list of Projects associated with the Subject.</li>
<li><b>vital_status:</b> Coded value indicating the state or condition of being living or deceased; also includes the case where the vital status is unknown.</li>
<li><b>days_to_death:</b> Number of days between the date used for index and the date from a person's date of death represented as a calculated number of days.</li>
<li><b>cause_of_death:</b> Coded value indicating the circumstance or condition that results in the death of the subject.</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our criteria, we can run our query against the researchsubject endpoint:

In [8]:
researchsubjectresults = myquery.researchsubject.run()
print(researchsubjectresults)

Getting results from database

Total execution time: 0
                        min 3.84 sec 3840 ms

Offset: 0
            Count: 100
            Total Row Count: 3716
            More pages: True

Now we see that our 2384 subjects have 3716 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [9]:
researchsubjectresults.to_dataframe()

researchsubject_id  \
0   4dd86ebd-ef16-4b2b-9ea0-5d1d7afef257   
1   be68cfb0-c47f-49a1-9e39-b0b3e57372fc   
2   99b34e59-a5d0-4a8b-b777-3a592c5bea1b   
3   f55673be-5796-49bb-aa0b-fed26281c36f   
4   9bfb5ec7-6fe1-443a-9eb2-75321b0da84a   
..                                   ...   
95  1827eb4b-38f3-45b5-a833-156ba748c784   
96  aa7502a9-5ffe-48c1-a1a8-3b07e83f78d7   
97  1ee67b45-5469-45e1-9dc0-8f68aa8d7cff   
98  34137fac-29bb-4b5e-964d-72a2237edf8e   
99  4447a969-e5c8-4291-b83c-53a0f7e77cbc   

                           researchsubject_identifier  \
0   [{'system': 'GDC', 'value': '4dd86ebd-ef16-4b2...   
1   [{'system': 'GDC', 'value': 'be68cfb0-c47f-49a...   
2   [{'system': 'GDC', 'value': '99b34e59-a5d0-4a8...   
3   [{'system': 'GDC', 'value': 'f55673be-5796-49b...   
4   [{'system': 'GDC', 'value': '9bfb5ec7-6fe1-443...   
..                                                ...   
95  [{'system': 'GDC', 'value': '1827eb4b-38f3-45b...   
96  [{'system': 'GDC', 'value': 'aa7502a9-5ffe-48c...   
97  [{'system': 'GDC', 'value': '1ee67b45-5469-45e...   
98  [{'system': 'GDC', 'value': '34137fac-29bb-4b5...   
99  [{'system': 'GDC', 'value': '4447a969-e5c8-429...   

   member_of_research_project primary_diagnosis_condition  \
0                   TCGA-DLBC     Mature B-Cell Lymphomas   
1                   GENIE-JHU              Neoplasms, NOS   
2                   GENIE-JHU              Neoplasms, NOS   
3                   GENIE-JHU              Neoplasms, NOS   
4   EXCEPTIONAL_RESPONDERS-ER              Neoplasms, NOS   
..                        ...                         ...   
95                    CPTAC-3                     Gliomas   
96                    CPTAC-3                     Gliomas   
97                    CPTAC-3                     Gliomas   
98                    CPTAC-3                     Gliomas   
99                    CPTAC-3                     Gliomas   

   primary_diagnosis_site       subject_id  
0                   Brain     TCGA-RQ-AAAT  
1                   Brain  GENIE-JHU-02615  
2                   Brain  GENIE-JHU-02402  
3                   Brain  GENIE-JHU-01988  
4                   Brain          ER-B1HR  
..                    ...              ...  
95                  Brain        C3N-00662  
96                  Brain        C3N-02784  
97                  Brain        C3N-02785  
98                  Brain        C3L-01156  
99                  Brain        C3L-03728  

[100 rows x 6 columns]

Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered near the end of the <a href="../BuildingACohort/#merging-results-across-endpoints">Cohort Building workflow</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. For CDA, this is case_id.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>member_of_research_project:</b> A reference to the Project(s) of which this ResearchSubject is a member.</li>
<li><b>primary_diagnosis_condition:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>primary_diagnosis_site:</b> The text term used to describe the primary site of disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This categorization groups cases into general categories. This attribute represents the primary site of disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>subject_id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about researchsubjects that have a diagnosis that meets our search criteria:

In [10]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 3.689 sec 3689 ms

diagnosis_id  \
0   05591a01-6481-5334-a7bb-107b22ef46da   
1   139314db-196b-43e9-bc57-a5ed9f5ea34b   
2   1433aaa9-8572-4abb-88eb-bfc12ebbcfd8   
3   2422c460-5d15-5ed9-8c03-f2fdf0c838c9   
4   3edb67f5-821a-5fda-b986-1b6a79ca2ca5   
..                                   ...   
95  7a2d4e0b-7492-5f85-ba22-311d363d97e7   
96  bbeeee2c-5444-57e2-a1d8-6da2e05ed38b   
97  c701f1e9-9043-5154-8b99-e09aa4b6828f   
98  d33f65e7-832c-4085-9593-b3028c35706c   
99  d36c239d-dfad-59ba-8ecf-6419b282f165   

                                 diagnosis_identifier  \
0   [{'system': 'GDC', 'value': '05591a01-6481-533...   
1   [{'system': 'GDC', 'value': '139314db-196b-43e...   
2   [{'system': 'GDC', 'value': '1433aaa9-8572-4ab...   
3   [{'system': 'GDC', 'value': '2422c460-5d15-5ed...   
4   [{'system': 'GDC', 'value': '3edb67f5-821a-5fd...   
..                                                ...   
95  [{'system': 'GDC', 'value': '7a2d4e0b-7492-5f8...   
96  [{'system': 'GDC', 'value': 'bbeeee2c-5444-57e...   
97  [{'system': 'GDC', 'value': 'c701f1e9-9043-515...   
98  [{'system': 'GDC', 'value': 'd33f65e7-832c-408...   
99  [{'system': 'GDC', 'value': 'd36c239d-dfad-59b...   

                primary_diagnosis  age_at_diagnosis morphology stage  \
0                Astrocytoma, NOS           11509.0     9400/3  None   
1           Mixed germ cell tumor               NaN     9085/3  None   
2                    Glioblastoma           19701.0     9440/3  None   
3                Astrocytoma, NOS           10960.0     9400/3  None   
4                    Mixed glioma           13903.0     9382/3  None   
..                            ...               ...        ...   ...   
95                   Glioblastoma           21617.0     9440/3  None   
96  Oligodendroglioma, anaplastic           13611.0     9451/3  None   
97  Oligodendroglioma, anaplastic           25809.0     9451/3  None   
98                      Germinoma               NaN     9064/3  None   
99        Astrocytoma, anaplastic           14205.0     9401/3  None   

           grade method_of_diagnosis           subject_id  \
0   not reported                None         TCGA-CS-4938   
1   Not Reported                None       GENIE-MDA-5982   
2   Not Reported                None            C3N-02769   
3   not reported                None         TCGA-DU-A76O   
4   not reported                None         TCGA-E1-5322   
..           ...                 ...                  ...   
95  not reported                None         TCGA-02-0004   
96  not reported                None         TCGA-HT-7470   
97  not reported                None         TCGA-DH-A669   
98  Not Reported                None  GENIE-MSK-P-0018993   
99  not reported                None         TCGA-DU-7298   

                      researchsubject_id  
0   334f715e-08dc-4a29-b8e4-b010b829c478  
1   f23eee6d-ec9e-4c2a-a60e-79c3a9ed8d4f  
2   2e653fba-5d9b-446f-af7f-a986e39ed713  
3   c56fa247-ad97-4dce-a2d3-9bdfc5dc7b1b  
4   5ea230a6-77aa-4aa5-867c-09651d481d8b  
..                                   ...  
95  3ac8f759-2df0-4ef5-913f-ce05b8dd8937  
96  fee2114e-f723-4374-91f8-f1dcc6b3eaee  
97  76e54ef6-1799-4fd9-bec3-0ea960e8461e  
98  edf00f04-3e93-41f0-9d5f-ce712553a6ba  
99  292cf158-dd47-44a1-912b-73626095a0f7  

[100 rows x 10 columns]


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>primary_diagnosis:</b> The diagnosis instance that qualified a subject for inclusion on a ResearchProject.</li>
  <li><b>age_at_diagnosis:</b> The age in days of the individual at the time of diagnosis.</li>
  <li><b>morphology:</b> Code that represents the histology of the disease using the third edition of the <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a>, published in 2000, used principally in tumor and cancer registries for coding the site (topography) and the histology (morphology) of neoplasms.</li>
  <li><b>stage:</b> The extent of a cancer in the body. Staging is usually based on the size of the tumor, whether lymph nodes contain cancer, and whether the cancer has spread from the original site to other parts of the body. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> The degree of abnormality of cancer cells, a measure of differentiation, the extent to which cancer cells are similar in appearance and function to healthy cells of the same tissue type. The degree of differentiation often relates to the clinical behavior of the particular tumor. Based on the microscopic findings, tumor grade is commonly described by one of four degrees of severity. Histopathologic grade of a tumor may be used to plan treatment and estimate the future course, outcome, and overall prognosis of disease. Certain types of cancers, such as soft tissue sarcoma, primary brain tumors, lymphomas, and breast have special grading systems.</li>
  <li><b>method_of_diagnosis:</b> The method used to confirm the patients malignant diagnosis.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on researchsubjects that have a given diagnosis that meets our search criteria:

In [11]:
treatmentresults = myquery.treatment.run()
treatmentresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 3.911 sec 3911 ms

treatment_id  \
0   2dca991d-6122-5171-ad38-bd8e0d72a28b   
1   31e9b3e3-9158-5c3d-a1aa-a34bcf4953ce   
2   35afc7ca-06d8-5946-a424-3e2e2626ac21   
3   35c3fb1b-9f1d-5550-9327-f8bffe01cf09   
4   395e3abf-cfba-59c4-bcbe-70c35a67276d   
..                                   ...   
95  248be3b9-8a63-5440-af93-337299f9c170   
96  2c448194-4608-5d8d-8337-1246082ed4c2   
97  4ad3b151-745f-5505-843b-37626243e1d0   
98  57a1318e-855b-5196-a83d-4532c5fc2860   
99  5b904cb2-623d-555c-ae6c-b8e0a66e906f   

                                 treatment_identifier  \
0   [{'system': 'GDC', 'value': '2dca991d-6122-517...   
1   [{'system': 'GDC', 'value': '31e9b3e3-9158-5c3...   
2   [{'system': 'GDC', 'value': '35afc7ca-06d8-594...   
3   [{'system': 'GDC', 'value': '35c3fb1b-9f1d-555...   
4   [{'system': 'GDC', 'value': '395e3abf-cfba-59c...   
..                                                ...   
95  [{'system': 'GDC', 'value': '248be3b9-8a63-544...   
96  [{'system': 'GDC', 'value': '2c448194-4608-5d8...   
97  [{'system': 'GDC', 'value': '4ad3b151-745f-550...   
98  [{'system': 'GDC', 'value': '57a1318e-855b-519...   
99  [{'system': 'GDC', 'value': '5b904cb2-623d-555...   

                 treatment_type treatment_outcome  days_to_treatment_start  \
0   Pharmaceutical Therapy, NOS              None                      NaN   
1   Pharmaceutical Therapy, NOS              None                      NaN   
2        Radiation Therapy, NOS              None                      NaN   
3        Radiation Therapy, NOS              None                      NaN   
4   Pharmaceutical Therapy, NOS              None                      NaN   
..                          ...               ...                      ...   
95       Radiation Therapy, NOS              None                      NaN   
96  Pharmaceutical Therapy, NOS              None                      NaN   
97  Pharmaceutical Therapy, NOS              None                      NaN   
98  Pharmaceutical Therapy, NOS              None                      NaN   
99  Pharmaceutical Therapy, NOS              None                      NaN   

    days_to_treatment_end therapeutic_agent treatment_anatomic_site  \
0                     NaN              None                    None   
1                     NaN              None                    None   
2                     NaN              None                    None   
3                     NaN              None                    None   
4                     NaN              None                    None   
..                    ...               ...                     ...   
95                    NaN              None                    None   
96                    NaN              None                    None   
97                    NaN              None                    None   
98                    NaN              None                    None   
99                    NaN              None                    None   

   treatment_effect treatment_end_reason number_of_cycles    subject_id  \
0              None                 None             None  TCGA-DB-A75L   
1              None                 None             None  TCGA-HT-8106   
2              None                 None             None  TCGA-HW-7493   
3              None                 None             None  TCGA-06-0939   
4              None                 None             None  TCGA-06-2570   
..              ...                  ...              ...           ...   
95             None                 None             None  TCGA-DU-A7TA   
96             None                 None             None  TCGA-06-0188   
97             None                 None             None  TCGA-06-0162   
98             None                 None             None  TCGA-06-5416   
99             None                 None             None  TCGA-06-0158   

                      researchsubject_id                          diagnosis_id  
0   ba62b5eb-e169-4b41-b949-e004d66e98a1  


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i> Medication administration or other treatment types. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>treatment_type:</b> The treatment type including medication/therapeutics or other procedures.</li>
  <li><b>treatment_outcome:</b> The final outcome of the treatment.</li>
  <li><b>days_to_treatment_start:</b> The timepoint at which the treatment started.</li>
  <li><b>days_to_treatment_end:</b>The timepoint at which the treatment ended. </li>
  <li><b>therapeutic_agent:</b> One or more therapeutic agents as part of this treatment.</li>
  <li><b>treatment_anatomic_site:</b> The anatomical site that the treatment targets.</li>
  <li><b>treatment_effect:</b>The effect of a treatment on the diagnosis or tumor. </li>
  <li><b>treatment_end_reason:</b>The reason the treatment ended. </li>
  <li><b>number_of_cycles:</b>The number of treatment cycles the subject received. </li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id:</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---




### specimen

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [12]:
specimenresults =  myquery.specimen.run()
specimenresults

Getting results from database

Total execution time: 0
                        min 3.929 sec 3929 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 39201
            More pages: True
            

Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as separate speciments. Let's look at a few:

In [13]:
specimenresults.to_dataframe()

specimen_id  \
0   00117d98-bd69-47a1-8d4b-9f30ff0220f0   
1   001a9bb8-8a78-440c-bcc2-02c6058b1c6d   
2   001ccfbe-71ac-42a9-9cc9-1c0f3b0f2358   
3   00d7fc45-b4d0-4137-93be-e1b2b00a2f08   
4   011913ab-a2a3-4ac9-8d41-c6733f4eb863   
..                                   ...   
95  39972bcf-8078-4989-80df-5fbe0736bb8a   
96  39ea05af-7b7b-56ee-8706-006111fd6949   
97  39f9ce44-944c-49b8-b215-05b117d5e62d   
98  3ace1801-8342-47f7-b749-8516e54d5843   
99  3b07c85a-ec0b-433c-95e2-fcdc963fa549   

                                  specimen_identifier  \
0   [{'system': 'GDC', 'value': '00117d98-bd69-47a...   
1   [{'system': 'GDC', 'value': '001a9bb8-8a78-440...   
2   [{'system': 'GDC', 'value': '001ccfbe-71ac-42a...   
3   [{'system': 'GDC', 'value': '00d7fc45-b4d0-413...   
4   [{'system': 'GDC', 'value': '011913ab-a2a3-4ac...   
..                                                ...   
95  [{'system': 'GDC', 'value': '39972bcf-8078-498...   
96  [{'system': 'GDC', 'value': '39ea05af-7b7b-56e...   
97  [{'system': 'GDC', 'value': '39f9ce44-944c-49b...   
98  [{'system': 'GDC', 'value': '3ace1801-8342-47f...   
99  [{'system': 'GDC', 'value': '3b07c85a-ec0b-433...   

   specimen_associated_project  days_to_collection primary_disease_type  \
0                     TCGA-LGG              4769.0              Gliomas   
1                     TCGA-GBM                 NaN              Gliomas   
2                     TCGA-GBM                 NaN              Gliomas   
3                     TCGA-GBM                 NaN              Gliomas   
4                     TCGA-GBM                 NaN              Gliomas   
..                         ...                 ...                  ...   
95                    TCGA-GBM                 NaN              Gliomas   
96                    TCGA-GBM                 0.0              Gliomas   
97                   HCMI-CMDC                 NaN              Gliomas   
98                    TCGA-LGG               924.0              Gliomas   
99                    TCGA-GBM                 NaN              Gliomas   

   anatomical_site  source_material_type specimen_type  \
0             None         Primary Tumor       aliquot   
1             None  Blood Derived Normal        sample   
2             None  Blood Derived Normal       aliquot   
3             None         Primary Tumor       aliquot   
4             None         Primary Tumor       aliquot   
..             ...                   ...           ...   
95            None  Blood Derived Normal       aliquot   
96            None         Primary Tumor       portion   
97            None       Recurrent Tumor       analyte   
98            None  Blood Derived Normal       aliquot   
99            None         Primary Tumor       aliquot   

                   derived_from_specimen         subject_id  \
0   25ef6ef4-4600-44f3-a17d-a03c5c77c9ae       TCGA-DU-A76L   
1                       initial specimen       TCGA-06-0184   
2   67e02b25-fcef-4c4e-be09-25c693e123b7       TCGA-19-2625   
3   5ec35ba5-f583-448e-bcb5-5b6a33f769a7       TCGA-08-0358   
4   a28a5b24-b0d0-450c-82f6-503fb65c082b       TCGA-12-1094   
..                                   ...                ...   
95  ae9aafab-ede4-4ccc-bf5e-1c69f1628250       TCGA-06-0159   
96  08b382c8-6d7b-485e-a22c-ce3b84bffe3f       TCGA-19-5952   
97  77827fa6-18a9-4e1a-b49f-800cec351fa8  HCM-BROD-0199-C71   
98  52135cc0-4e77-4dcc-b9ce-cd5e70ffa287       TCGA-TQ-A7RJ   
99  d8e68043-63d8-46e5-9a8e-c420b732a699       TCGA-02-0046   

                      researchsubject_id  
0   395798c5-3136-4a30-8557-a67a3abdecaf  
1   1f48f010-98fe-4b5a-b96a-14fb25eff23f  
2   f6b2f52b-284d-4bde-9a3d-a0f458d07f08  
3   ec504558-e96a-44e9-835a-7b12680681ed  
4   dca3a016-5e89-4850-80bf-e78dc19a2453  
..                                   ...  
95  5f8f66a0-4068-4e9d-82fc-4fe1fc3f63aa  
96  ef36cbd1-5d7d-4756-a628-b5339d41e228  
97  149a8565-e0c5-4474-a693-d44f1b445c0c  
98  df78e63d-4387-4177-b24f-f0


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i><i>Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</i>
 A given specimen will have only a single subject ID and a single researchsubject ID</i>
    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>associated_project:</b> The Project associated with the specimen.</li>
  <li><b>days_to_collection:</b> The number of days from the index date to either the date a sample was collected for a specific study or project, or the date a patient underwent a procedure (e.g. surgical resection) yielding a sample that was eventually used for research.</li>
  <li><b>primary_disease_type:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
  <li><b>anatomical_site:</b> Per GDC Dictionary, the text term that represents the name of the primary disease site of the submitted tumor sample; recommend dropping tumor; biospecimen_anatomic_site.</li>
  <li><b>source_material_type:</b> The general kind of material from which the specimen was derived, indicating the physical nature of the source material.</li>
  <li><b>specimen_type:</b> The high-level type of the specimen, based on its how it has been derived from the original extracted sample. One of: analyte, aliquot, portion, sample, or slide.</li>
  <li><b>derived_from_specimen:</b> A source/parent specimen from which this one was directly derived.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, regardless of whether they are attached to subjects, researchsubjects or specimens: 

In [14]:
myquery.file.run()

Getting results from database

Total execution time: 0
                        min 3.919 sec 3919 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 4275020
            More pages: True
            

In [15]:
fileresults = myquery.file.run()
fileresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.038 sec 4038 ms

file_id  \
0   3aa300b0-5528-4fed-8326-d03dceb2839f   
1   63aad47d-f0c6-4fd5-8efe-0ba417bafc00   
2   c0922d50-ac2d-41f6-8a10-e417fbc37e36   
3   7b1b9967-2178-413a-b0c5-1957385e492d   
4   c5bdfb90-dcda-413a-b9c7-70d0dded69a4   
..                                   ...   
95  60da3038-53bd-4abf-82fd-b2b1f3fd8146   
96  a98de346-0a25-447f-bbbf-1eeffcbfd09a   
97  e4071190-4449-49ca-92fb-865887d139eb   
98  f2f47e86-6713-4825-bea6-d73fc0e4f889   
99  2284bab1-f537-4634-b58b-1971707b66f1   

                                      file_identifier  \
0   [{'system': 'IDC', 'value': '3aa300b0-5528-4fe...   
1   [{'system': 'IDC', 'value': '63aad47d-f0c6-4fd...   
2   [{'system': 'IDC', 'value': 'c0922d50-ac2d-41f...   
3   [{'system': 'IDC', 'value': '7b1b9967-2178-413...   
4   [{'system': 'IDC', 'value': 'c5bdfb90-dcda-413...   
..                                                ...   
95  [{'system': 'IDC', 'value': '60da3038-53bd-4ab...   
96  [{'system': 'IDC', 'value': 'a98de346-0a25-447...   
97  [{'system': 'IDC', 'value': 'e4071190-4449-49c...   
98  [{'system': 'IDC', 'value': 'f2f47e86-6713-482...   
99  [{'system': 'IDC', 'value': '2284bab1-f537-463...   

                                           label data_category data_type  \
0   idc/3aa300b0-5528-4fed-8326-d03dceb2839f.dcm       Imaging      None   
1   idc/63aad47d-f0c6-4fd5-8efe-0ba417bafc00.dcm       Imaging      None   
2   idc/c0922d50-ac2d-41f6-8a10-e417fbc37e36.dcm       Imaging      None   
3   idc/7b1b9967-2178-413a-b0c5-1957385e492d.dcm       Imaging      None   
4   idc/c5bdfb90-dcda-413a-b9c7-70d0dded69a4.dcm       Imaging      None   
..                                           ...           ...       ...   
95  idc/60da3038-53bd-4abf-82fd-b2b1f3fd8146.dcm       Imaging      None   
96  idc/a98de346-0a25-447f-bbbf-1eeffcbfd09a.dcm       Imaging      None   
97  idc/e4071190-4449-49ca-92fb-865887d139eb.dcm       Imaging      None   
98  idc/f2f47e86-6713-4825-bea6-d73fc0e4f889.dcm       Imaging      None   
99  idc/2284bab1-f537-4634-b58b-1971707b66f1.dcm       Imaging      None   

   file_format     file_associated_project  \
0        DICOM           acrin_fmiso_brain   
1        DICOM                      ivygap   
2        DICOM                    tcga_gbm   
3        DICOM                    tcga_gbm   
4        DICOM                   cptac_gbm   
..         ...                         ...   
95       DICOM                      ivygap   
96       DICOM                    tcga_gbm   
97       DICOM                   cptac_gbm   
98       DICOM          acrin_dsc_mr_brain   
99       DICOM  qin_gbm_treatment_response   

                                              drs_uri  byte_size checksum  \
0   drs://dg.4DFC:3aa300b0-5528-4fed-8326-d03dceb2...        NaN     None   
1   drs://dg.4DFC:63aad47d-f0c6-4fd5-8efe-0ba417ba...        NaN     None   
2   drs://dg.4DFC:c0922d50-ac2d-41f6-8a10-e417fbc3...        NaN     None   
3   drs://dg.4DFC:7b1b9967-2178-413a-b0c5-1957385e...        NaN     None   
4   drs://dg.4DFC:c5bdfb90-dcda-413a-b9c7-70d0dded...        NaN     None   
..                                                ...        ...      ...   
95  drs://dg.4DFC:60da3038-53bd-4abf-82fd-b2b1f3fd...        NaN     None   
96  drs://dg.4DFC:a98de346-0a25-447f-bbbf-1eeffcbf...        NaN     None   
97  drs://dg.4DFC:e4071190-4449-49ca-92fb-865887d1...        NaN     None   
98  drs://dg.4DFC:f2f47e86-6713-4825-bea6-d73fc0e4...        NaN     None   
99  drs://dg.4DFC:2284bab1-f537-4634-b58b-1971707b...        NaN     None   

   data_modality imaging_modality dbgap_accession_number  \
0        Imaging               MR                   None   
1        Imaging               MR                   None   
2        Imaging               MR                   None   
3        Imaging               MR                   None   
4        Imaging               MR                   None   
..           ...              ...                    ...   
95      

As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to chain your queries together, which we cover in the next tutorial <a href="../Chaining">Chaining Endpoints</a>. Or to build a more complex query using <a href="../Operators">Operators</a>. 

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the <a href="../DataSummaries">Data Summaries tutorial</a> to try it.



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>label:</b> Short name or abbreviation for dataset. Maps to rdfs:label.</li>
  <li><b>data_catagory:</b> Broad categorization of the contents of the data file.</li>
  <li><b>data_type:</b> Specific content type of the data file.</li>
  <li><b>file_format:</b> Format of the data files.</li>
  <li><b>associated_project:</b> A reference to the Project(s) of which this ResearchSubject is a member. The associated_project may be embedded using the ref definition or may be a reference to the id for the Project - or a URI expressed as a string to an existing entity.</li>
  <li><b>drs_uri:</b> A string of characters used to identify a resource on the Data Repo Service(DRS). Can be used to retrieve this specific file from a server.</li>
  <li><b>byte_size:</b> Size of the file in bytes. Maps to dcat:byteSize.</li>
  <li><b>checksum:</b> The md5 value for the file. A digit representing the sum of the correct digits in a piece of stored or transmitted digital data, against which later comparisons can be made to detect errors in the data.</li>
  <li><b>data_modality:</b> Data modality describes the biological nature of the information gathered as the result of an activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging".</li>
  <li><b>imaging_modality:</b> An imaging modality describes the imaging equipment and/or method used to acquire certain structural or functional information about the body. These include but are not limited to computed tomography (CT) and magnetic resonance imaging (MRI). Taken from the DICOM standard.</li>
  <li><b>dbgap_accession_number:</b> The dbgap accession number for the project.</li>
</ul>  

</div>
    
---

### mutation

The file endpoint returns molecular data about specific mutations that meet our search criteria. Search is currently limited to the TCGA-READ project from GDC:

In [16]:
myquery.mutation.run()

Getting results from database

Total execution time: 0
                        min 4.113 sec 4113 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 904
            More pages: True
            

In [17]:
mutationresults = myquery.mutation.run()
mutationresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 3.976 sec 3976 ms

project_short_name  case_barcode sample_barcode_tumor  \
0            TCGA-GBM  TCGA-02-2486     TCGA-02-2486-01A   
1            TCGA-GBM  TCGA-06-2561     TCGA-06-2561-01A   
2            TCGA-GBM  TCGA-12-1602     TCGA-12-1602-01A   
3            TCGA-GBM  TCGA-19-2624     TCGA-19-2624-01A   
4            TCGA-GBM  TCGA-19-4068     TCGA-19-4068-01A   
..                ...           ...                  ...   
95           TCGA-LGG  TCGA-HT-A61C     TCGA-HT-A61C-01A   
96           TCGA-LGG  TCGA-P5-A5ET     TCGA-P5-A5ET-01A   
97           TCGA-LGG  TCGA-S9-A7R3     TCGA-S9-A7R3-01A   
98           TCGA-LGG  TCGA-TM-A84R     TCGA-TM-A84R-01A   
99           TCGA-GBM  TCGA-06-0189     TCGA-06-0189-01A   

   sample_barcode_normal         aliquot_barcode_tumor  \
0       TCGA-02-2486-10A  TCGA-02-2486-01A-01D-1494-08   
1       TCGA-06-2561-10A  TCGA-06-2561-01A-02D-1494-08   
2       TCGA-12-1602-10A  TCGA-12-1602-01A-01W-0643-08   
3       TCGA-19-2624-10A  TCGA-19-2624-01A-01D-1495-08   
4       TCGA-19-4068-10A  TCGA-19-4068-01A-01D-1353-08   
..                   ...                           ...   
95      TCGA-HT-A61C-10A  TCGA-HT-A61C-01A-11D-A29Q-08   
96      TCGA-P5-A5ET-10A  TCGA-P5-A5ET-01A-11D-A27K-08   
97      TCGA-S9-A7R3-10A  TCGA-S9-A7R3-01A-11D-A34J-08   
98      TCGA-TM-A84R-10A  TCGA-TM-A84R-01A-21D-A36O-08   
99      TCGA-06-0189-10A  TCGA-06-0189-01A-01D-1491-08   

          aliquot_barcode_normal Hugo_Symbol  Entrez_Gene_Id Center  \
0   TCGA-02-2486-10A-01D-1494-08     FAM150A          389658     BI   
1   TCGA-06-2561-10A-01D-1494-08       NETO2           81831     BI   
2   TCGA-12-1602-10A-01W-0643-08    TMEM132D          121256     BI   
3   TCGA-19-2624-10A-01D-1495-08      ZNF727          442319     BI   
4   TCGA-19-4068-10A-01D-1353-08     FAM171B          165215     BI   
..                           ...         ...             ...    ...   
95  TCGA-HT-A61C-10A-01D-A29Q-08       SLIT2            9353     BI   
96  TCGA-P5-A5ET-10A-01D-A27N-08        DLL1           28514     BI   
97  TCGA-S9-A7R3-10A-01D-A34M-08       GHRHR            2692     BI   
98  TCGA-TM-A84R-10A-01D-A367-08        UBR3          130507     BI   
99  TCGA-06-0189-10A-01D-1491-08     WBSCR27          155368     BI   

   NCBI_Build  ...                        tumor_bam_uuid  \
0      GRCh38  ...  4850f0ea-f177-48c6-80b8-35cf390d4ea0   
1      GRCh38  ...  282cbea5-1456-4b97-8da8-9a80112246a1   
2      GRCh38  ...  a80a834c-8d31-4949-b01c-a92707c974dc   
3      GRCh38  ...  74fca1ea-2dbe-42d5-8e93-24d65785299f   
4      GRCh38  ...  9d64cfc1-f26d-415e-b89f-fd6a56d4db0e   
..        ...  ...                                   ...   
95     GRCh38  ...  f22a68fc-9133-4dab-9ade-d1b41dee6e55   
96     GRCh38  ...  b653ba7c-e021-4282-aa5c-4f69b9b9c34d   
97     GRCh38  ...  ee7ba9e7-8b99-46cc-8364-c7686bc84f36   
98     GRCh38  ...  6ad5609c-492f-4263-b314-86ec0b704565   
99     GRCh38  ...  ef31cb62-6b0e-4833-b42c-247f19fbf202   

                         normal_bam_uuid  \
0   e5bc7b12-a3cf-4fd1-8061-962b381051ca   
1   78fb3cc0-66f3-49e3-a850-072e030d5a43   
2   dde11930-80f9-4bb2-9db7-73b82b321a45   
3   285ce22f-4360-475b-9541-9f4e619623cb   
4   dff9919b-2523-4daa-a0ed-5878e0583200   
..                                   ...   
95  9b506c49-fe2a-45ad-bbbd-ce90dac3728d   
96  f147ca5f-1524-4c95-bdaf-3d0511f6fb83   
97  0594b028-724b-41b5-86fb-2e53c651f06a   
98  b041636b-adfa-4ce4-bacd-3856547b2cb5   
99  5c7fdf0f-a2f0-4b97-938b-048f5726ba9c   

                                 case_id GDC_FILTER       COSMIC MC3_Overlap  \
0   79a68b32-29a3-47bf-9b2b-643ec7323d73       None  COSM3413058        True   
1   d6a48eb1-7c20-4ae3-ae35-b6b641238ef1       None         None        True   
2   8ac5b22f-3925-47a9-8d09-d819c2e191e5   wga_pair         None        True   
3   2f4a127f-101a-4192-b3e8-f9be2c8648bc       None         None        True   
4   ef646014-78a9-43af-b6fa-85fa288cd51b       None         None        True   
..